# Japanese simulations analysis

In this notebook, we are going to analyze the datasets coming from the japanese simulations of Nishimichi.
Let us start importing the necessary packages

In [ ]:
using Statistics
using BlindedChallenge
#using JSON3
using Plots, LinearAlgebra, BenchmarkTools, Random
using Base: @kwdef
using BSON: @save
using BSON: @load
using BSON
using LaTeXStrings
using Distributions
using SimpleChains
using Static
#using NPZ
#using QuadGK
#using StatsBase: sample, Weights
using StatsPlots
using ForwardDiff
using LinearAlgebra
using Statistics
using ProgressMeter
using Turing
using Pathfinder
using Optim
using Transducers

In [ ]:
using Revise
using Effort

First: let us decide if we are going to use lagrangian or optimal resummation

In [ ]:
resum = "optimal"
if resum == "lagrangian"
    println("You choose lagrangian resummation!")
elseif resum == "optimal"
    println("You choose optimal resummation!")
else
    error("You didn't choose a viable resummation!")
end

In [ ]:
rescale_cov = false
if rescale_cov
    println("You decided to rescale the covariance cov!")
    scaling_factor = 4
    scaling_name = "_scaled_"
else
    println("You decided not to rescale the covariance!")
    scaling_factor = 1
    scaling_name = ""
end;

Let us load the trained Effort emulators for the blinded challenge.

In [ ]:
if resum == "lagrangian"
    println("You choose lagrangian resummation!")
    Mono_Emu = BSON.load("/home/marcobonici/Desktop/CosmologicalEmulators/trained_emulators/trained_effort_emulators/blinded_challenge/PyBird_061_10000_lagrangian_guido_spectra_check/emulator_PyBird_w0wanucdm_monopole.bson")[:Pℓ]
    Quad_Emu = BSON.load("/home/marcobonici/Desktop/CosmologicalEmulators/trained_emulators/trained_effort_emulators/blinded_challenge/PyBird_061_10000_lagrangian_guido_spectra_check/emulator_PyBird_w0wanucdm_quadrupole.bson")[:Pℓ];
elseif resum == "optimal"
    println("You choose optimal resummation!")
    Mono_Emu = BSON.load("/home/marcobonici/Desktop/CosmologicalEmulators/trained_emulators/trained_effort_emulators/blinded_challenge/PyBird_061_10000_optiresum_final/emulator_PyBird_w0wanucdm_monopole.bson")[:Pℓ]
    Quad_Emu = BSON.load("/home/marcobonici/Desktop/CosmologicalEmulators/trained_emulators/trained_effort_emulators/blinded_challenge/PyBird_061_10000_optiresum_final/emulator_PyBird_w0wanucdm_quadrupole.bson")[:Pℓ];
else
    error("You didn't choose a viable resummation!")
end;

# The likelihood

In [ ]:
function theory(θ, n, Mono_Emu, Quad_Emu)
    # θ[1:3] cosmoparams, ln_10_As, H0, ΩM
    # θ[4:9] bias
    # θ[10:11] stoch
    #f = fN(θ[3], 0.61)
    f = Effort._f_z(0.61, θ[3], -1., 0.);
    #conversion from c2-c4 to b2-b4
    b2 = (θ[5]+θ[7])/√2
    b4 = (θ[5]-θ[7])/√2
    my_θ = deepcopy(θ)
    my_θ[8] /= (0.7^2)
    my_θ[9] /= (0.7^2)
    n_bar = 3e-4 #value  that has been suggested by Guido himself
    k_bins = Effort.create_bin_edges(BlindedChallenge.k_grid)
    #stoch_0, stoch_2 = Effort.get_stoch_terms(0, θ[10], θ[11], n_bar, k_grid)
    stoch_0, stoch_2 = Effort.get_stoch_terms_binned_efficient(0., my_θ[10], my_θ[11], n_bar, k_bins)
    return vcat((Effort.get_Pℓ(my_θ[1:3], vcat(my_θ[4], b2, my_θ[6], b4, my_θ[8:9] , 0), f, Mono_Emu) .+ stoch_0)[1:n],
                (Effort.get_Pℓ(my_θ[1:3], vcat(my_θ[4], b2, my_θ[6], b4, my_θ[8:9] , 0), f, Quad_Emu) .+ stoch_2)[1:n])
end

In [ ]:
@benchmark theory(ones(11), 20, Mono_Emu, Quad_Emu)

# Turing & Pathfinder

We are now ready to write our loglikelihood!
We are going to employ Turing.jl, a Probabilistic Programming Language (PPL) written in Julia.
Our model can be conveniently written using the `@model` macro.
It is quite easy to write the priors with this formalism. The priors employed in this analysis are the same used by Guido in its analysis
- $b_1$ $\sim$ $\mathrm{Uniform}$ $(0, 4)$
- $c_2$ $\sim$ $\mathrm{Uniform}$ $(-4, 4)$
- $b_3$ $\sim$ $\mathcal{N}(0,10)$ 
- $c_4$ $\sim$ $\mathcal{N}(0,2)$
- $c_\mathrm{ct}$ $\sim$ $\mathcal{N}(0,4)$
- $c_\mathrm{r1}$ $\sim$ $\mathcal{N}(0,8)$
- $c_{\epsilon,m}$ $\sim$ $\mathcal{N}(0,2)$
- $c_{\epsilon,q}$ $\sim$ $\mathcal{N}(0,4)$

In [ ]:
@model function pplmodel(data, cov, n, Mono_Emu, Quad_Emu)
    ln10As ~ Uniform(2.9, 3.15)
    H0     ~ Uniform(62.,68.)
    ΩM     ~ Uniform(0.30, 0.34)
    b1     ~ Uniform(0., 4.)
    c2     ~ Uniform(-4., 4.)
    b3     ~ Normal(0., 10.)
    c4     ~ Normal(0., 2.)
    cct    ~ Normal(0., 4.)
    cr1    ~ Normal(0., 8.)
    cϵm    ~ Normal(0., 2.)
    cϵq    ~ Normal(0., 4.)

    θ = [ln10As, H0, ΩM, b1, c2, b3, c4, cct, cr1, cϵm, cϵq]

    prediction = theory(θ, n, Mono_Emu, Quad_Emu)

    data ~ MvNormal(prediction, cov)
    return nothing
end

n = 20

data_20, k_20, cov_20, yerror_Mono_20, yerror_Quad_20 = BlindedChallenge.create_data(n)

model_20 = pplmodel(data_20, cov_20.*scaling_factor, 20, Mono_Emu, Quad_Emu);

In [ ]:
result_multi_20 = multipathfinder(model_20, 1000; nruns = 8, executor = Transducers.PreferParallel())

In [ ]:
nsteps  = 1000
nadapts = 250
nchains = 16

In [ ]:
init_params_20 = collect.(eachrow(result_multi_20.draws_transformed.value[1:nchains, :, 1]));

In [ ]:
chains_20 = sample(model_20, Turing.NUTS(nadapts, 0.65), MCMCThreads(), nsteps, nchains; init_params = init_params_20)

In [ ]:
p = plot(chains_20)
savefig("chains_turing_"*resum*scaling_name*"_Pathfinder_20.pdf")
savefig("chains_turing_"*resum*scaling_name*"_Pathfinder_20.png")
p